In [ ]:
# Input filename and theta. It is assumed that the filenames are same.
# For e.g., if filename is 'abc', implies 'abc.pepa' is the pepa model description file as per assumption in paper,
# 'abc.statespace' is the state-space generated with PEPA with state numbers only included and the separator is chosen
# as ',' and 'abc.generator' is the rate transition matrix generated using PEPA with the same method for state-space.

filename='badge'
theta='99999'

In [ ]:
f1=filename+'.statespace'
f2=filename+'.generator'
f3=filename+'.pepa'

In [ ]:
# Read the respective files.

with open(f1,'r') as f:
    l1=f.readlines()
f.close()
with open(f2,'r') as f:
    l2=f.readlines()
f.close()
with open(f3,'r') as f:
    l3=f.readlines()
f.close()
#print(l1,l2,l3)

In [ ]:
# Infer the states.

states=[]
for i in range(len(l1)):
    states.append('s'+str(i))
#print(states)

In [ ]:
action=[]
initial='s0'
tran={}

In [ ]:
# Infer actions between states using the PEPA specification file.

for i in range(len(l3)):
    a=l3[i].rstrip()
    if a:
        if ',' in a and '=' in a and ';' in a and '.' in a and a[0].isupper() or a[0]=='#':
            a=a.replace(' ','')
            a=a.replace('#','')
            a=a.replace(';','')
            b=a.split('=')[-1]
            b=b.split('+')
            #print(b)
            if a.split('=')[0] not in tran:
                tran[a.split('=')[0]]={}
                for j in range(len(b)):
                    c=b[j].split(').')
                    key=c[-1]
                    val=''
                    for k in range(c[0].index('(')+1,c[0].index(',')):
                        val+=c[0][k]
                    if val not in action:
                        action.append(val)
                    tran[a.split('=')[0]][key]=val

            else:
                for j in range(len(b)):
                    c=b[j].split(').')
                    key=c[-1]
                    val=''
                    for k in range(c[0].index('(')+1,c[0].index(',')):
                        val+=c[0][k]
                    if val not in action:
                        action.append(val)
                    tran[a.split('=')[0]][key]=val
            
#print(tran)

In [ ]:
# Infer labels for each state.

state_label={}
state_labels=[]
for i in range(len(l1)):
    a=l1[i].split('{')[-1]
    a=a.rstrip()
    a=a.replace('}','')
    a=a.split(',')
    state_label['s'+str(i)]=[]
    for j in range(len(a)):
        if a[j] not in state_labels:
            state_labels.append(a[j])
        state_label['s'+str(i)].append(a[j])        
#print(state_label)
#print(state_labels)

In [ ]:
# Create the transition list using information from the three files.

tran_list=[]
live=[]
for i in range(len(l2)):
    tlm=[]
    a=l2[i].rstrip()
    a=a.split(',')
    ori='s'+str(int(a[0])-1)
    if ori not in live:
        live.append(ori)
    des='s'+str(int(a[1])-1)
    rate=a[-1]
    #print(ori,des,rate)
    flag=0
    act=[]
    for i in range(len(state_label[ori])):
        if state_label[ori][i]!=state_label[des][i]:
            acti=tran[state_label[ori][i]][state_label[des][i]]
            if acti not in act:
                act.append(acti)
            #print(state_label[ori][i],state_label[des][i],act)
            flag=1
            #break
    if flag==0:
        print('enter')
        act='tau'
    tlm.append(ori)
    if len(act)==1:
        tlm.append(act[0]+'.'+des)
    else:
        print(act)
        print("TRANSFORMATION WILL NOT WORK!!!!!!")
    tlm.append(rate)
    tran_list.append(tlm)
#print(tran_list)

In [ ]:
# Identify if deadlocks exist in the model.

deadlock=list((set(live)-set(states)))
#print(deadlock)

In [ ]:
# Copying transitions to begin transforming into SCTMC for PRISM.

prism_pr_adj={}
for i in range(len(tran_list)):
    if str(tran_list[i][0]) not in prism_pr_adj:
        prism_pr_adj[str(tran_list[i][0])]={}
        prism_pr_adj[str(tran_list[i][0])][str(tran_list[i][1])]=str(tran_list[i][-1])
    else:
        prism_pr_adj[str(tran_list[i][0])][str(tran_list[i][1])]=str(tran_list[i][-1])
#print(prism_pr_adj)

In [ ]:
# Constructing the machine readable files for PRISM.

labels={}
mapping={}
ats_adj={}
transition_count=1
ctr=0
for x in prism_pr_adj:
    a=x
    b=prism_pr_adj[x]
    #print(a,b)
    if a not in labels:
        labels[a]='bot'
    if a not in mapping:
        mapping[a]=a
    for y in b:
        if y not in mapping and y.split('.')[0]!='tau':
            mapping[y]='s'+str(len(states)+ctr)
            ctr+=1
        if y not in labels and y.split('.')[0]!='tau':
            labels[mapping[y]]=y.split('.')[0]
    if x not in ats_adj:
        ats_adj[x]={}
        for y in prism_pr_adj[x]:
            if y.split('.')[0]!='tau':
                ats_adj[x][mapping[y]]=prism_pr_adj[x][y]
                transition_count+=1
            else:
                ats_adj[x][y.split('.')[-1]]=prism_pr_adj[x][y]
                transition_count+=1
for x,y in mapping.items():
    if x!=y:
        ats_adj[y]={x.split('.')[-1]:theta}
        transition_count+=1
for x in deadlock:
    labels[x]='bot'
    mapping[x]=x
# Creating the Labels (.lab) File.
Lab={'"init"':0,'"deadlock"':1,'"bot"':2}
ctr=3
for i in labels.values():
    if '"'+i+'"' not in Lab:
        Lab['"'+i+'"']=ctr
        ctr+=1
ats_labels=''
for i in Lab:
    ats_labels+=str(Lab[i])+'='+str(i)+' '
ats_labels=ats_labels[:-1]
sorted_lab1={}
for i in labels:
    sorted_lab1[int(i.split('s')[-1])]=labels[i]
sorted_lab2 = {key: value for key, value in sorted(sorted_lab1.items())}
for i in sorted_lab2:
    if 's'+str(i)!=initial and 's'+str(i) not in deadlock:
        ats_labels+='\n'+str(i)+':'+' '+str(Lab['"'+sorted_lab2[i]+'"'])
    elif 's'+str(i)==initial:
        ats_labels+='\n'+str(i)+':'+' 0'+' '+str(Lab['"'+sorted_lab2[i]+'"'])
    else:
        ats_labels+='\n'+str(i)+':'+' 1'+' '+str(Lab['"'+sorted_lab2[i]+'"'])
#print(ats_labels)
# Creating the State and Transitions File.
tran=str(len(mapping))+' '+str(transition_count)
sorted_adj1={}
for i in ats_adj:
    sorted_adj1[int(i.split('s')[-1])]={}
    for j in ats_adj[i]:
        sorted_adj1[int(i.split('s')[-1])][int(j.split('s')[-1])]=ats_adj[i][j]
sorted_adj2 = {key: value for key, value in sorted(sorted_adj1.items())}
#print(sorted_adj2)
for i in sorted_adj2:
    for j in sorted_adj2[i]:
        if sorted_adj2[i][j]=='1.0':
            tran+='\n'+str(i)+' '+str(j)+' '+'1'
        else:
            tran+='\n'+str(i)+' '+str(j)+' '+str(sorted_adj2[i][j])
#print(tran)
state_file='(s)'
for i in range(len(mapping)):
    state_file+='\n'+str(i)+': ('+str(i)+')'

    #print(state_file)

In [ ]:
# Writing to respective files for PRISM.

with open('ats'+filename+'.tra','w') as file:
    file.write(tran)
file.close()
with open('ats'+filename+'.lab','w') as file:
    file.write(ats_labels)
file.close()
with open('ats'+filename+'.sta','w') as file:
    file.write(state_file)
file.close()

In [ ]:
# Generating a human readable PRISM code.

code3=''
code3+='\n\n ctmc \n\t const int theta='+str(theta)+';\n\n'+'\n\t module ats_ACTMC \n\n //Variables\n'
code3+='s:[0..'+str(len(mapping)-1)+'] init '+initial.split('s')[-1]
code3+=';\n'
for a,b in ats_adj.items():
    code3+='\n [] s='+a.split('s')[-1]+' ->'
    for c in b:
        if b[c]!=theta:
            code3+=' '+str(b[c])+":(s'="+c.split('s')[-1]+')+'
        else:
            code3+=' '+'theta'+":(s'="+c.split('s')[-1]+')+'
    code3=code3[:-1]
    code3+=';'
for x in deadlock:
    code3+='\n [] s='+x.split('s')[-1]+" -> theta:(s'="+x.split('s')[-1]+');'
code3+='\n\n\t endmodule \n\n\t //Labels'
code3+='\n label "Initial" = (s='+initial.split('s')[-1]+');'
if deadlock:
    code3+='\n label "Final" = '
    for x in deadlock:
        code3+='(s='+x.split('s')[-1]+')|'
    code3=code3[:-1]
    code3+=';'
code3+='\n label "bot"= '
for i in labels:
    if labels[i]=='bot':
        code3+='(s='+i.split('s')[-1]+')|'
code3=code3[:-1]
code3+=';'
for i in range(len(action)):
    flag=0
    for j in labels:
        if labels[j]==action[i]:
            flag=1
            break
    if flag==1:
        code3+='\n label "'+action[i]+'"= '
        for j in labels:
            if labels[j]==action[i]:
                code3+='(s='+j.split('s')[-1]+')|'
        code3=code3[:-1]
        code3+=';'
    else:
        continue

code3+='\n\n\t //Labels-2'
for i in state_labels:
    code3+='\n label "'+i+'" = '
    for j in states:
        if i in state_label[j]:
            code3+='(s='+j.split('s')[-1]+')|'
    code3=code3[:-1]
    code3+=';'

code3+='\n\n //Rewards' 
code3+='\n rewards "original_step"\n'
for i in labels:
    if labels[i]=='bot':
        code3+='(s='+i.split('s')[-1]+')'+'|'
code3=code3[:-1]
code3+=': 1;\n endrewards\n'
for i in range(len(action)):
    flag2=0
    for j in labels:
        if labels[j]==action[i]:
            flag2=1
            break
    if flag2==1:
        code3+='\n rewards "'+action[i]+'"\n'
        for j in labels:
            if labels[j]==action[i]:
                code3+='(s='+j.split('s')[-1]+')'+'|'
        code3=code3[:-1]
        code3+=': 1;\n endrewards\n'
    else:
        continue
code3+='\n\n //State Mapping'
for x,y in mapping.items():
    code3+='\n // '+x+' -> '+y
#print(code3)

In [ ]:
# Writing the code as a PRISM file.

with open('ats'+filename+'.prism','w') as file:
    file.write(code3)
file.close()